# LangChain Models: LLMs, Chat Models & Providers

## Introduction

LangChain supports multiple model types and providers, allowing you to write provider-agnostic code that can switch between OpenAI, Anthropic, Google, and others with minimal changes.

### Model Types

1. **Chat Models** (Recommended): Message-based interface (System, Human, AI messages)
2. **LLMs** (Legacy): Simple text in, text out (being phased out)
3. **Embeddings**: Convert text to vector representations

---

## Chat Models vs LLMs

### Chat Models (Modern Standard)

- Input: List of messages (SystemMessage, HumanMessage, AIMessage)
- Output: AIMessage object
- Use for: GPT-4, Claude, Gemini, and all modern conversational models

### LLMs (Legacy)

- Input: String
- Output: String
- Use for: Older completion models (GPT-3, etc.)

**Recommendation**: Always use Chat Models for new projects.

---

## Supported Providers (2025)

| Provider | Package | Models | Best For |
|----------|---------|--------|----------|
| OpenAI | `langchain-openai` | GPT-4, GPT-3.5 | General purpose, function calling |
| Anthropic | `langchain-anthropic` | Claude 3/4 | Long context, reasoning, safety |
| Google | `langchain-google-genai` | Gemini Pro | Multimodal, free tier |
| Cohere | `langchain-cohere` | Command R+ | RAG, search |
| Mistral | `langchain-mistral` | Mistral Large | Open source alternative |
| Local | `langchain-ollama` | Llama, Mistral | Privacy, offline |

---

## Setup: Multiple Providers

In [ ]:
import os
from getpass import getpass

# Set API keys for different providers
# You can choose one or multiple providers

# OpenAI
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API Key (or press Enter to skip): ") or "skip"

# Anthropic
if not os.getenv("ANTHROPIC_API_KEY"):
    os.environ["ANTHROPIC_API_KEY"] = getpass("Enter Anthropic API Key (or press Enter to skip): ") or "skip"

# Google
if not os.getenv("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass("Enter Google API Key (or press Enter to skip): ") or "skip"

print("\nAPI keys configured!")

---

## Example 1: OpenAI Chat Models

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

# Initialize OpenAI model
llm_openai = ChatOpenAI(
    model="gpt-4",
    temperature=0.7,
    max_tokens=500
)

# Create messages
messages = [
    SystemMessage(content="You are a helpful Python programming assistant."),
    HumanMessage(content="Explain list comprehensions in one sentence.")
]

# Invoke the model
response = llm_openai.invoke(messages)
print(f"OpenAI Response:\n{response.content}")
print(f"\nMetadata: {response.response_metadata}")

---

## Example 2: Anthropic Claude Models

In [ ]:
from langchain_anthropic import ChatAnthropic

# Initialize Anthropic model
llm_claude = ChatAnthropic(
    model="claude-sonnet-4-5-20250929",
    temperature=0.7,
    max_tokens=500
)

# Same messages as before!
response = llm_claude.invoke(messages)
print(f"Claude Response:\n{response.content}")
print(f"\nMetadata: {response.response_metadata}")

---

## Example 3: Google Gemini Models

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize Google Gemini model
llm_gemini = ChatGoogleGenerativeAI(
    model="gemini-pro",
    temperature=0.7,
    max_tokens=500
)

# Same messages as before!
response = llm_gemini.invoke(messages)
print(f"Gemini Response:\n{response.content}")

### Key Insight: Provider Independence

Notice how all three providers:
- Accept the same message format
- Use the same `.invoke()` method
- Return AIMessage objects

This makes switching providers trivial!

---

## Model Parameters

### Common Parameters (All Providers)

```python
llm = ChatOpenAI(
    model="gpt-4",              # Model name
    temperature=0.7,            # 0.0 (deterministic) to 2.0 (creative)
    max_tokens=500,             # Maximum tokens in response
    timeout=30,                 # Request timeout in seconds
    max_retries=2,              # Number of retries on failure
)
```

### Temperature Guide

| Temperature | Use Case | Example |
|-------------|----------|----------|
| 0.0 - 0.3 | Deterministic, factual | Code generation, Q&A |
| 0.4 - 0.7 | Balanced | General conversation |
| 0.8 - 1.2 | Creative | Story writing, brainstorming |
| 1.3 - 2.0 | Very creative | Experimental, artistic |

---

## Example 4: Temperature Comparison

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

prompt = "Write a creative name for a coffee shop"

temperatures = [0.0, 0.5, 1.0, 1.5]

for temp in temperatures:
    llm = ChatOpenAI(model="gpt-4", temperature=temp)
    response = llm.invoke(prompt)
    print(f"Temperature {temp}: {response.content}")

---

## Example 5: Streaming Responses

Stream tokens as they're generated for better UX:

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4")

print("Streaming response:\n")
for chunk in llm.stream("Write a haiku about Python programming"):
    print(chunk.content, end="", flush=True)

print("\n\nDone!")

---

## Example 6: Async Streaming

For non-blocking streaming:

In [ ]:
import asyncio
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4")

async def async_stream_example():
    print("Async streaming:\n")
    async for chunk in llm.astream("Explain asyncio in Python"):
        print(chunk.content, end="", flush=True)
    print("\n\nDone!")

# Run in Jupyter
await async_stream_example()

---

## Example 7: Multi-Model Comparison

Compare responses from different providers:

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI

def compare_models(prompt: str):
    """Compare same prompt across multiple providers."""
    
    models = {
        "OpenAI GPT-4": ChatOpenAI(model="gpt-4", temperature=0.7),
        "Anthropic Claude": ChatAnthropic(model="claude-sonnet-4-5-20250929", temperature=0.7),
        "Google Gemini": ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)
    }
    
    print(f"Prompt: {prompt}\n")
    print("=" * 80)
    
    for name, model in models.items():
        try:
            response = model.invoke(prompt)
            print(f"\n{name}:")
            print("-" * 80)
            print(response.content)
        except Exception as e:
            print(f"\n{name}: Error - {e}")

# Try it
compare_models("What are the key benefits of using design patterns in software development?")

---

## Example 8: Token Usage Tracking

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.callbacks import get_openai_callback

llm = ChatOpenAI(model="gpt-4")

# Track token usage and cost
with get_openai_callback() as cb:
    response1 = llm.invoke("What is Python?")
    response2 = llm.invoke("What is JavaScript?")
    
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost:.4f}")

---

## Example 9: Model Fallback

Try multiple models with automatic fallback:

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import RunnableWithFallbacks

# Primary model
primary = ChatOpenAI(model="gpt-4", temperature=0.7)

# Fallback models
fallback1 = ChatAnthropic(model="claude-sonnet-4-5-20250929")
fallback2 = ChatOpenAI(model="gpt-3.5-turbo")

# Create fallback chain
model_with_fallbacks = primary.with_fallbacks([fallback1, fallback2])

# If GPT-4 fails, tries Claude, then GPT-3.5
response = model_with_fallbacks.invoke("Explain Python decorators")
print(response.content)

---

## Example 10: Caching Responses

Cache LLM responses to save costs and improve performance:

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
import time

# Enable caching
set_llm_cache(InMemoryCache())

llm = ChatOpenAI(model="gpt-4")

prompt = "What is the capital of France?"

# First call - hits the API
start = time.time()
response1 = llm.invoke(prompt)
time1 = time.time() - start
print(f"First call: {time1:.2f}s")
print(response1.content)

# Second call - uses cache
start = time.time()
response2 = llm.invoke(prompt)
time2 = time.time() - start
print(f"\nSecond call (cached): {time2:.2f}s")
print(response2.content)

print(f"\nSpeedup: {time1/time2:.1f}x faster")

### Cache Types

```python
# In-memory (development)
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())

# SQLite (persistent)
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

# Redis (production)
from langchain.cache import RedisCache
set_llm_cache(RedisCache(redis_url="redis://localhost:6379"))
```

---

## Example 11: Embeddings

Convert text to vector representations (for RAG, similarity search):

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_anthropic import AnthropicEmbeddings

# OpenAI embeddings
embeddings_openai = OpenAIEmbeddings(model="text-embedding-3-small")

# Embed a single text
text = "LangChain makes building LLM applications easy"
vector = embeddings_openai.embed_query(text)

print(f"Vector dimension: {len(vector)}")
print(f"First 5 values: {vector[:5]}")

# Embed multiple texts
texts = ["Python is great", "JavaScript is popular", "Rust is fast"]
vectors = embeddings_openai.embed_documents(texts)
print(f"\nEmbedded {len(vectors)} documents")

### Embedding Models Comparison

| Provider | Model | Dimensions | Cost | Best For |
|----------|-------|------------|------|----------|
| OpenAI | text-embedding-3-small | 1536 | Low | General purpose |
| OpenAI | text-embedding-3-large | 3072 | Medium | High accuracy |
| Cohere | embed-english-v3.0 | 1024 | Medium | Search, RAG |
| HuggingFace | all-MiniLM-L6-v2 | 384 | Free | Local, offline |

---

## Example 12: Function Calling (OpenAI)

Let the model call functions with structured outputs:

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field

# Define function schema using Pydantic
class GetWeather(BaseModel):
    """Get the current weather for a location."""
    location: str = Field(description="The city and state, e.g., San Francisco, CA")
    unit: str = Field(default="celsius", description="Temperature unit")

# Bind function to model
llm = ChatOpenAI(model="gpt-4")
llm_with_tools = llm.bind_tools([GetWeather])

# Model will decide to call the function
response = llm_with_tools.invoke("What's the weather in Paris?")
print(response)

# Check if model wants to call a function
if response.tool_calls:
    tool_call = response.tool_calls[0]
    print(f"\nFunction: {tool_call['name']}")
    print(f"Arguments: {tool_call['args']}")

---

## Provider-Specific Features

### OpenAI

- ✅ Function calling
- ✅ JSON mode
- ✅ Vision (GPT-4V)
- ✅ Best ecosystem support

### Anthropic Claude

- ✅ 200K token context window
- ✅ Superior reasoning
- ✅ Strong safety guardrails
- ✅ Constitutional AI

### Google Gemini

- ✅ Multimodal (text, images, video)
- ✅ Free tier available
- ✅ Fast inference
- ✅ Integrated with Google services

---

## Best Practices

### ✅ Do

1. **Use Chat Models** for all new projects (not legacy LLMs)
2. **Set timeouts** to prevent hanging requests
3. **Implement retries** for production reliability
4. **Cache responses** when appropriate
5. **Track token usage** to monitor costs
6. **Use fallbacks** for high availability

### ❌ Don't

1. **Don't hardcode API keys** in code (use environment variables)
2. **Don't ignore rate limits** (implement backoff)
3. **Don't use high temperature** for factual tasks
4. **Don't forget error handling** (network issues, API errors)
5. **Don't batch without limits** (respect rate limits)

---

## Practice Exercises

In [ ]:
# Exercise 1: Create a function that tries multiple providers
# If one fails, automatically try the next

def get_response_with_fallback(prompt: str) -> str:
    """Try multiple providers, return first successful response."""
    from langchain_openai import ChatOpenAI
    from langchain_anthropic import ChatAnthropic
    
    models = [
        ChatOpenAI(model="gpt-4"),
        ChatAnthropic(model="claude-sonnet-4-5-20250929"),
        ChatOpenAI(model="gpt-3.5-turbo")
    ]
    
    for i, model in enumerate(models):
        try:
            response = model.invoke(prompt)
            print(f"Success with model {i+1}")
            return response.content
        except Exception as e:
            print(f"Model {i+1} failed: {e}")
            continue
    
    return "All models failed"

# Test it
result = get_response_with_fallback("Explain Python generators")
print(f"\nResult: {result}")

In [ ]:
# Exercise 2: Compare response times across providers
import time
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

def benchmark_models(prompt: str):
    """Benchmark response time for different models."""
    models = {
        "GPT-4": ChatOpenAI(model="gpt-4"),
        "GPT-3.5": ChatOpenAI(model="gpt-3.5-turbo"),
        "Claude": ChatAnthropic(model="claude-sonnet-4-5-20250929")
    }
    
    results = {}
    
    for name, model in models.items():
        try:
            start = time.time()
            response = model.invoke(prompt)
            elapsed = time.time() - start
            results[name] = {
                "time": elapsed,
                "response_length": len(response.content)
            }
        except Exception as e:
            results[name] = {"error": str(e)}
    
    return results

# Test it
results = benchmark_models("Explain the factory pattern in 2 sentences")
for model, data in results.items():
    if "error" in data:
        print(f"{model}: Error - {data['error']}")
    else:
        print(f"{model}: {data['time']:.2f}s ({data['response_length']} chars)")

In [ ]:
# Exercise 3: Create a cost estimator
# Calculate approximate cost for a given prompt and number of completions

def estimate_cost(prompt: str, num_completions: int, model_name: str = "gpt-4") -> dict:
    """Estimate cost for multiple completions."""
    from langchain_openai import ChatOpenAI
    from langchain.callbacks import get_openai_callback
    
    llm = ChatOpenAI(model=model_name)
    
    with get_openai_callback() as cb:
        # Make one call to estimate
        llm.invoke(prompt)
        
        # Extrapolate
        estimated_tokens = cb.total_tokens * num_completions
        estimated_cost = cb.total_cost * num_completions
    
    return {
        "model": model_name,
        "completions": num_completions,
        "estimated_tokens": estimated_tokens,
        "estimated_cost_usd": estimated_cost
    }

# Test it
estimate = estimate_cost("Explain Python asyncio", 100, "gpt-4")
print(f"Estimated cost for {estimate['completions']} completions:")
print(f"Tokens: {estimate['estimated_tokens']}")
print(f"Cost: ${estimate['estimated_cost_usd']:.2f}")

---

## Key Takeaways

### ✅ What We Learned

1. **Provider Independence**: Same interface for OpenAI, Anthropic, Google
2. **Chat Models**: Modern message-based interface (recommended)
3. **Temperature Control**: 0.0 (deterministic) to 2.0 (creative)
4. **Streaming**: Better UX with token-by-token responses
5. **Caching**: Save costs and improve performance
6. **Fallbacks**: High availability with multiple providers
7. **Token Tracking**: Monitor usage and costs
8. **Embeddings**: Convert text to vectors for RAG

### 📚 Next Steps

- **langchain_prompts.ipynb**: Advanced prompt engineering
- **langchain_lcel.ipynb**: Master the Expression Language
- **langchain_rag.ipynb**: Build RAG applications

---

## Resources

- [OpenAI Models](https://platform.openai.com/docs/models)
- [Anthropic Claude](https://www.anthropic.com/claude)
- [Google Gemini](https://ai.google.dev/)
- [LangChain Model Documentation](https://python.langchain.com/docs/integrations/chat/)

---

**Next Notebook**: `langchain_prompts.ipynb` - Prompt engineering techniques